In [4]:
from git import Repo
import os
import pandas as pd
import numpy as np
import mysql.connector



# Clonning from the repository

In [5]:
repo_path = "https://github.com/PhonePe/pulse.git"
clone_path = r"E:\pulse\data"

if not os.path.exists(clone_path):
    os.makedirs(clone_path)

repo_path = os.path.join(clone_path, os.path.basename(repo_url).removesuffix('.git').title())

Repo.clone_from(repo_url, repo_path)

directory = os.path.join(repo_path, 'data')
print(directory)

NameError: name 'repo_url' is not defined

# Renaming sub-directories and Extracting necessary paths

In [6]:
# Function to rename messy state names in a proper format

def rename(directory):
    for root, dirs, files in os.walk(directory):
        if 'state' in dirs:
            state_dir = os.path.join(root, 'state')
            for state_folder in os.listdir(state_dir):
                # rename the state folder
                old_path = os.path.join(state_dir, state_folder)
                new_path = os.path.join(state_dir, state_folder.title().replace('-', ' ').replace('&', 'and'))
                os.rename(old_path, new_path)
    print("Renamed all sub-directories successfully")
                
# Function to extract all paths that has sub-directory in the name of 'state'

def extract_paths(directory):
    path_list = []
    for root, dirs, files in os.walk(directory):
        if os.path.basename(root) == 'state':
            path_list.append(root.replace('\\', '/'))
    return path_list

In [7]:
rename(directory)

NameError: name 'directory' is not defined

In [8]:
state_directories = extract_paths(directory)
state_directories

NameError: name 'directory' is not defined

# Creating dataframes from cloned json files

# 1.Aggregate Transaction

In [80]:
state_path = state_directories[0]
state_list = os.listdir(state_path)
agg_trans_dict = {
                  'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [],
                  'Transaction_count': [], 'Transaction_amount': []
                  }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for transaction_data in df['data']['transactionData']:
                    
                    type = transaction_data['name']
                    count = transaction_data['paymentInstruments'][0]['count']
                    amount = transaction_data['paymentInstruments'][0]['amount']
                    
                    # Appending to agg_trans_dict
                    
                    agg_trans_dict['State'].append(state)
                    agg_trans_dict['Year'].append(year)
                    agg_trans_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    agg_trans_dict['Transaction_type'].append(type)
                    agg_trans_dict['Transaction_count'].append(count)
                    agg_trans_dict['Transaction_amount'].append(amount)
            except:
                pass
            
agg_trans_df = pd.DataFrame(agg_trans_dict)

In [184]:
agg_trans_df

,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount,Region
0,Andaman and Nicobar Islands,2018,1,Recharge & bill payments,4200,1.845307e+06,Southern Region
1,Andaman and Nicobar Islands,2018,1,Peer-to-peer payments,1871,1.213866e+07,Southern Region
2,Andaman and Nicobar Islands,2018,1,Merchant payments,298,4.525072e+05,Southern Region
3,Andaman and Nicobar Islands,2018,1,Financial Services,33,1.060142e+04,Southern Region
4,Andaman and Nicobar Islands,2018,1,Others,256,1.846899e+05,Southern Region
...,...,...,...,...,...,...,...
3589,West Bengal,2022,4,Peer-to-peer payments,184380244,6.202222e+11,Eastern Region
3590,West Bengal,2022,4,Merchant payments,171667404,1.408077e+11,Eastern Region
3591,West Bengal,2022,4,Recharge & bill payments,48921147,2.602663e+10,Eastern Region
3592,West Bengal,2022,4,Financial Services,268388,2.611229e+08,Eastern Region


In [194]:
import mysql.connector as mysql
from mysql.connector import Error

def create_aggregated_transcation():
    
    try:
        connection = mysql.connect(
                            host = 'localhost',
                            database = 'phonepe_pulse',
                            user = 'root',
                            password = 'krishaan28920*'
                        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            print('MySQL connected successfully!')
        
        cursor.execute('DROP TABLE IF EXISTS aggregated_transcation')

        cursor.execute('''CREATE TABLE IF NOT EXISTS aggregated_transcation(
                                            state             VARCHAR(255),
                                            year              INT,
                                            quater            VARCHAR(255),
                                            transacion_type   VARCHAR(255),
                                            transacion_count  INT,
                                            transacion_amount FLOAT(50,3),
                                            region            VARCHAR(255))'''
                      )

        query = '''
                INSERT INTO aggregated_transcation (state, year, quater, transacion_type, transacion_count, transacion_amount, region)
                VALUES (%s, %s, %s, %s, %s, %s, %s)

                '''
        aggregated_transcation = agg_trans_df

        for _, row in aggregated_transcation.iterrows():
            values = tuple(row)
            cursor.execute(query, values)
            connection.commit()
            
    except Error as e:
        print(f'Error occured: {str(e)}')
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print('MySQL connection closed.')
            


In [195]:
 create_aggregated_transcation()

MySQL connected successfully!
MySQL connection closed.


# 2. Aggregate User

In [81]:
state_path = state_directories[1]
state_list = os.listdir(state_path)
agg_user_dict = {
                 'State': [], 'Year': [], 'Quarter': [], 'Brand': [],
                 'Transaction_count': [], 'Percentage': []
                 }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for user_data in df['data']['usersByDevice']:

                    brand = user_data['brand']
                    count = user_data['count']
                    percent = user_data['percentage']
                    
                    # Appending to agg_user_dict
                    
                    agg_user_dict['State'].append(state)
                    agg_user_dict['Year'].append(year)
                    agg_user_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    agg_user_dict['Brand'].append(brand)
                    agg_user_dict['Transaction_count'].append(count)
                    agg_user_dict['Percentage'].append(percent)
            except:
                pass

agg_user_df = pd.DataFrame(agg_user_dict)

In [196]:
agg_user_df

,State,Year,Quarter,Brand,Transaction_count,Percentage,Region
0,Andaman and Nicobar Islands,2018,1,Xiaomi,1665,0.247033,Southern Region
1,Andaman and Nicobar Islands,2018,1,Samsung,1445,0.214392,Southern Region
2,Andaman and Nicobar Islands,2018,1,Vivo,982,0.145697,Southern Region
3,Andaman and Nicobar Islands,2018,1,Oppo,501,0.074332,Southern Region
4,Andaman and Nicobar Islands,2018,1,OnePlus,332,0.049258,Southern Region
...,...,...,...,...,...,...,...
6727,West Bengal,2022,1,Lenovo,330017,0.015056,Eastern Region
6728,West Bengal,2022,1,Infinix,284678,0.012987,Eastern Region
6729,West Bengal,2022,1,Asus,280347,0.012790,Eastern Region
6730,West Bengal,2022,1,Apple,277752,0.012671,Eastern Region


In [199]:
def create_aggregated_user():
    
    try:
        connection = mysql.connect(
                            host = 'localhost',
                            database = 'phonepe_pulse',
                            user = 'root',
                            password = 'krishaan28920*'
                        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            print('MySQL connected successfully!')
        
        cursor.execute('DROP TABLE IF EXISTS aggregated_user')

        cursor.execute('''CREATE TABLE IF NOT EXISTS aggregated_user(
                                            state             VARCHAR(255),
                                            year              INT,
                                            quater            VARCHAR(255),
                                            brand             VARCHAR(255),
                                            brand_count       INT,
                                            brand_percentage  FLOAT,
                                            region            VARCHAR(255))'''
                      )

        query = '''
                INSERT INTO aggregated_user (state, year, quater, brand, brand_count, brand_percentage, region)
                VALUES (%s, %s, %s, %s, %s, %s, %s)

                '''
        aggregated_user = agg_user_df

        for _, row in aggregated_user.iterrows():
            values = tuple(row)
            cursor.execute(query, values)
            connection.commit()
            
    except Error as e:
        print(f'Error occured: {str(e)}')
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print('MySQL connection closed.')
            
   

In [200]:
create_aggregated_user()

MySQL connected successfully!
MySQL connection closed.


# 3. Map Transaction

In [82]:
state_path = state_directories[2]
state_list = os.listdir(state_path)
map_trans_dict = {
                    'State': [], 'Year': [], 'Quarter': [], 'District': [],
                    'Transaction_count': [], 'Transaction_amount': []
                    }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for transaction_data in df['data']['hoverDataList']:
                   
                    district = transaction_data['name']
                    count = transaction_data['metric'][0]['count']
                    amount = transaction_data['metric'][0]['amount']
                    
                    # Appending to map_trans_dict
                    
                    map_trans_dict['State'].append(state)
                    map_trans_dict['Year'].append(year)
                    map_trans_dict['Quarter'].append(int(quarter.removesuffix('.json'))) 
                    map_trans_dict['District'].append(district.removesuffix(' district').title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    map_trans_dict['Transaction_count'].append(count)
                    map_trans_dict['Transaction_amount'].append(amount)
            except:
                pass

map_trans_df = pd.DataFrame(map_trans_dict)

In [201]:
map_trans_df

,State,Year,Quarter,District,Transaction_count,Transaction_amount,Region
0,Andaman and Nicobar Islands,2018,1,North and Middle Andaman,442,9.316631e+05,Southern Region
1,Andaman and Nicobar Islands,2018,1,South Andaman,5688,1.256025e+07,Southern Region
2,Andaman and Nicobar Islands,2018,1,Nicobars,528,1.139849e+06,Southern Region
3,Andaman and Nicobar Islands,2018,2,North and Middle Andaman,825,1.317863e+06,Southern Region
4,Andaman and Nicobar Islands,2018,2,South Andaman,9395,2.394824e+07,Southern Region
...,...,...,...,...,...,...,...
14631,West Bengal,2022,4,Nadia,12690126,2.804568e+10,Eastern Region
14632,West Bengal,2022,4,Birbhum,7617444,1.614650e+10,Eastern Region
14633,West Bengal,2022,4,Purba Medinipur,14484229,3.309949e+10,Eastern Region
14634,West Bengal,2022,4,Maldah,12492746,2.721861e+10,Eastern Region


In [212]:
def create_map_transcation():
    
    try:
        connection = mysql.connect(
                            host = 'localhost',
                            database = 'phonepe_pulse',
                            user = 'root',
                            password = 'krishaan28920*'
                        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            print('MySQL connected successfully!')
        
        cursor.execute('DROP TABLE IF EXISTS map_transcation')

        cursor.execute('''CREATE TABLE IF NOT EXISTS map_transcation(
                                            state             VARCHAR(255),
                                            year              INT,
                                            quater            VARCHAR(255),
                                            district          VARCHAR(255),
                                            transacion_count  INT,
                                            transacion_amount DECIMAL(20,5),
                                            region            VARCHAR(255))'''
                      )

        query = '''
                INSERT INTO map_transcation (state, year, quater, district, transacion_count, transacion_amount, region)
                VALUES (%s, %s, %s, %s, %s, %s, %s)

                '''
        map_transcation = map_trans_df

        for _, row in map_transcation.iterrows():
            values = tuple(row)
            cursor.execute(query, values)
            connection.commit()
            
    except Error as e:
        print(f'Error occured: {str(e)}')
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print('MySQL connection closed.')
            

In [213]:
 create_map_transcation()

MySQL connected successfully!
MySQL connection closed.


# 4. Map User

In [83]:
state_path = state_directories[3]
state_list = os.listdir(state_path)
map_user_dict = {
                 'State': [], 'Year': [], 'Quarter': [], 'District': [],
                 'Registered_users': [], 'App_opens': []
                 }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for district, user_data in df['data']['hoverData'].items():
                    
                    reg_user_count = user_data['registeredUsers']
                    app_open_count = user_data['appOpens']
                    
                    # Appending to map_user_dict
                    
                    map_user_dict['State'].append(state)
                    map_user_dict['Year'].append(year)
                    map_user_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    map_user_dict['District'].append(district.removesuffix(' district').title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    map_user_dict['Registered_users'].append(reg_user_count)
                    map_user_dict['App_opens'].append(app_open_count)
            except:
                pass

map_user_df = pd.DataFrame(map_user_dict)

In [214]:
map_user_df

,State,Year,Quarter,District,Registered_users,App_opens,Region
0,Andaman and Nicobar Islands,2018,1,North and Middle Andaman,632,0,Southern Region
1,Andaman and Nicobar Islands,2018,1,South Andaman,5846,0,Southern Region
2,Andaman and Nicobar Islands,2018,1,Nicobars,262,0,Southern Region
3,Andaman and Nicobar Islands,2018,2,North and Middle Andaman,911,0,Southern Region
4,Andaman and Nicobar Islands,2018,2,South Andaman,8143,0,Southern Region
...,...,...,...,...,...,...,...
14635,West Bengal,2022,4,Nadia,1359420,33853990,Eastern Region
14636,West Bengal,2022,4,Birbhum,855236,20950662,Eastern Region
14637,West Bengal,2022,4,Purba Medinipur,1346908,38278506,Eastern Region
14638,West Bengal,2022,4,Maldah,954892,29023743,Eastern Region


In [215]:
def create_map_user():
    
    try:
        connection = mysql.connect(
                            host = 'localhost',
                            database = 'phonepe_pulse',
                            user = 'root',
                            password = 'krishaan28920*'
                        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            print('MySQL connected successfully!')
        
        cursor.execute('DROP TABLE IF EXISTS map_user')

        cursor.execute('''CREATE TABLE IF NOT EXISTS map_user(
                                            state             VARCHAR(255),
                                            year              INT,
                                            quater            VARCHAR(255),
                                            district          VARCHAR(255),
                                            registered_user   INT,
                                            app_opening       INT,
                                            region            VARCHAR(255))'''
                      )

        query = '''
                INSERT INTO map_user (state, year, quater, district, registered_user, app_opening, region)
                VALUES (%s, %s, %s, %s, %s, %s, %s)

                '''
        map_user = map_user_df

        for _, row in map_user.iterrows():
            values = tuple(row)
            cursor.execute(query, values)
            connection.commit()
            
    except Error as e:
        print(f'Error occured: {str(e)}')
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print('MySQL connection closed.')

In [216]:
create_map_user()

MySQL connected successfully!
MySQL connection closed.


# 5. Top Transaction District-wise 

In [84]:
state_path = state_directories[4]
state_list = os.listdir(state_path)
top_trans_dist_dict = {
                        'State': [], 'Year': [], 'Quarter': [], 'District': [],
                        'Transaction_count': [], 'Transaction_amount': []
                        }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for district_data in df['data']['districts']:
                    
                    name = district_data['entityName']
                    count = district_data['metric']['count']
                    amount = district_data['metric']['amount']
                    
                    # Appending to top_trans_dist_dict
                    
                    top_trans_dist_dict['State'].append(state)
                    top_trans_dist_dict['Year'].append(year)
                    top_trans_dist_dict['Quarter'].append(int(quarter.removesuffix('.json')))                    
                    top_trans_dist_dict['District'].append(name.title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    top_trans_dist_dict['Transaction_count'].append(count)
                    top_trans_dist_dict['Transaction_amount'].append(amount)
            except:
                pass

top_trans_dist_df = pd.DataFrame(top_trans_dist_dict)

In [217]:
top_trans_dist_df

,State,Year,Quarter,District,Transaction_count,Transaction_amount,Region
0,Andaman and Nicobar Islands,2018,1,South Andaman,5688,1.256025e+07,Southern Region
1,Andaman and Nicobar Islands,2018,1,Nicobars,528,1.139849e+06,Southern Region
2,Andaman and Nicobar Islands,2018,1,North and Middle Andaman,442,9.316631e+05,Southern Region
3,Andaman and Nicobar Islands,2018,2,South Andaman,9395,2.394824e+07,Southern Region
4,Andaman and Nicobar Islands,2018,2,Nicobars,1120,3.072437e+06,Southern Region
...,...,...,...,...,...,...,...
5915,West Bengal,2022,4,South Twenty Four Parganas,15651650,3.373698e+10,Eastern Region
5916,West Bengal,2022,4,Purba Medinipur,14484229,3.309949e+10,Eastern Region
5917,West Bengal,2022,4,Hooghly,13931352,2.755409e+10,Eastern Region
5918,West Bengal,2022,4,Howrah,13350090,2.793786e+10,Eastern Region


In [218]:
def create_top_transcation():
    
    try:
        connection = mysql.connect(
                            host = 'localhost',
                            database = 'phonepe_pulse',
                            user = 'root',
                            password = 'krishaan28920*'
                        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            print('MySQL connected successfully!')
        
        cursor.execute('DROP TABLE IF EXISTS top_transcation')

        cursor.execute('''CREATE TABLE IF NOT EXISTS top_transcation(
                                            state             VARCHAR(255),
                                            year              INT,
                                            quater            VARCHAR(255),
                                            district          VARCHAR(255),
                                            transacion_count  INT,
                                            transacion_amount FLOAT(50,3),
                                            region            VARCHAR(255))'''
                      )

        query = '''
                INSERT INTO top_transcation (state, year, quater, district, transacion_count, transacion_amount, region)
                VALUES (%s, %s, %s, %s, %s, %s, %s)

                '''
        top_transcation = top_trans_dist_df

        for _, row in top_transcation.iterrows():
            values = tuple(row)
            cursor.execute(query, values)
            connection.commit()
            
    except Error as e:
        print(f'Error occured: {str(e)}')
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print('MySQL connection closed.')
            

In [220]:
create_top_transcation()

MySQL connected successfully!
MySQL connection closed.


# 6. Top Transaction Pincode-wise

In [85]:
state_path = state_directories[4]
state_list = os.listdir(state_path)
top_trans_pin_dict = {
                        'State': [], 'Year': [], 'Quarter': [], 'Pincode': [],
                        'Transaction_count': [], 'Transaction_amount': []
                        }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for regional_data in df['data']['pincodes']:
                    
                    name = regional_data['entityName']
                    count = regional_data['metric']['count']
                    amount = regional_data['metric']['amount']
                    
                    # Appending to top_trans_pin_dict
                    
                    top_trans_pin_dict['State'].append(state)
                    top_trans_pin_dict['Year'].append(year)
                    top_trans_pin_dict['Quarter'].append(int(quarter.removesuffix('.json')))                    
                    top_trans_pin_dict['Pincode'].append(name)
                    top_trans_pin_dict['Transaction_count'].append(count)
                    top_trans_pin_dict['Transaction_amount'].append(amount)
            except:
                pass

top_trans_pin_df = pd.DataFrame(top_trans_pin_dict)

In [221]:
top_trans_pin_df

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount,Region
0,Andaman and Nicobar Islands,2018,1,744101,1622,2.769298e+06,Southern Region
1,Andaman and Nicobar Islands,2018,1,744103,1223,2.238042e+06,Southern Region
2,Andaman and Nicobar Islands,2018,1,744102,969,3.519060e+06,Southern Region
3,Andaman and Nicobar Islands,2018,1,744105,685,1.298561e+06,Southern Region
4,Andaman and Nicobar Islands,2018,1,744104,340,1.039715e+06,Southern Region
...,...,...,...,...,...,...,...
7134,West Bengal,2022,4,722101,2900058,5.748321e+09,Eastern Region
7135,West Bengal,2022,4,700135,2471048,3.527457e+09,Eastern Region
7136,West Bengal,2022,4,732101,2407008,5.052109e+09,Eastern Region
7137,West Bengal,2022,4,700091,2348447,2.176640e+09,Eastern Region


In [234]:
def create_top_transcation_pincode():
    
    try:
        connection = mysql.connect(
                            host = 'localhost',
                            database = 'phonepe_pulse',
                            user = 'root',
                            password = 'krishaan28920*'
                        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            print('MySQL connected successfully!')
        
        cursor.execute('DROP TABLE IF EXISTS top_transcation_pincode')

        cursor.execute('''CREATE TABLE IF NOT EXISTS top_transcation_pincode(
                                            state             VARCHAR(255),
                                            pincode           INT,
                                            year              INT,
                                            quater            VARCHAR(255),
                                            transacion_count  INT,
                                            transacion_amount FLOAT(50,3),
                                            region            VARCHAR(255))'''
                      )

        query = '''
                INSERT INTO top_transcation_pincode (state, pincode, year, quater, transacion_count, transacion_amount, region)
                VALUES (%s, %s, %s, %s, %s, %s, %s)

                '''
        top_transcation_pincode = top_trans_pin_df

        for _, row in top_transcation_pincode.iterrows():
            values = tuple(row)
            cursor.execute(query, values)
            connection.commit()
            
    except Error as e:
        print(f'Error occured: {str(e)}')
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print('MySQL connection closed.')
            

In [235]:
create_top_transcation_pincode()

MySQL connected successfully!
MySQL connection closed.


# 7. Top User District-wise

In [86]:
state_path = state_directories[5]
state_list = os.listdir(state_path)
top_user_dist_dict = {
                        'State': [], 'Year': [], 'Quarter': [],
                        'District': [], 'Registered_users': []
                        }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for district_data in df['data']['districts']:
                    
                    name = district_data['name']
                    count = district_data['registeredUsers']
                    
                    # Appending to top_user_dist_dict
                    
                    top_user_dist_dict['State'].append(state)
                    top_user_dist_dict['Year'].append(year)
                    top_user_dist_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    top_user_dist_dict['District'].append(name.title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    top_user_dist_dict['Registered_users'].append(count)
            except:
                pass

top_user_dist_df = pd.DataFrame(top_user_dist_dict)

In [225]:
top_user_dist_df

,State,Year,Quarter,District,Registered_users,Region
0,Andaman and Nicobar Islands,2018,1,South Andaman,5846,Southern Region
1,Andaman and Nicobar Islands,2018,1,North and Middle Andaman,632,Southern Region
2,Andaman and Nicobar Islands,2018,1,Nicobars,262,Southern Region
3,Andaman and Nicobar Islands,2018,2,South Andaman,8143,Southern Region
4,Andaman and Nicobar Islands,2018,2,North and Middle Andaman,911,Southern Region
...,...,...,...,...,...,...
5915,West Bengal,2022,4,Howrah,1422011,Eastern Region
5916,West Bengal,2022,4,Nadia,1359420,Eastern Region
5917,West Bengal,2022,4,Purba Medinipur,1346908,Eastern Region
5918,West Bengal,2022,4,Paschim Medinipur,1217113,Eastern Region


In [226]:
def create_top_user():
    
    try:
        connection = mysql.connect(
                            host = 'localhost',
                            database = 'phonepe_pulse',
                            user = 'root',
                            password = 'krishaan28920*'
                        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            print('MySQL connected successfully!')
        
        cursor.execute('DROP TABLE IF EXISTS top_user')

        cursor.execute('''CREATE TABLE IF NOT EXISTS top_user(
                                            state             VARCHAR(255),
                                            year              INT,
                                            quater            VARCHAR(255),
                                            district          VARCHAR(255),
                                            registered_user   INT,
                                            region            VARCHAR(255))'''
                      )

        query = '''
                INSERT INTO top_user (state, year, quater, district, registered_user, region)
                VALUES (%s, %s, %s, %s, %s, %s)

                '''
        top_user = top_user_dist_df

        for _, row in top_user.iterrows():
            values = tuple(row)
            cursor.execute(query, values)
            connection.commit()
            
    except Error as e:
        print(f'Error occured: {str(e)}')
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print('MySQL connection closed.')
            

In [227]:
create_top_user()

MySQL connected successfully!
MySQL connection closed.


# 8. Top User Pincode-wise

In [87]:
state_path = state_directories[5]
state_list = os.listdir(state_path)
top_user_pin_dict = {
                        'State': [], 'Year': [], 'Quarter': [],
                        'Pincode': [], 'Registered_users': []
                        }

for state in state_list:
    year_path = state_path + '/' + state + '/'
    year_list = os.listdir(year_path)
    
    for year in year_list:
        quarter_path = year_path + year + '/'
        quarter_list = os.listdir(quarter_path)
        
        for quarter in quarter_list:
            json_path = quarter_path + quarter
            df = pd.read_json(json_path)
            
            try:
                for regional_data in df['data']['pincodes']:
                    
                    name = regional_data['name']
                    count = regional_data['registeredUsers']
                    
                    # Appending to top_user_pin_dict
                    
                    top_user_pin_dict['State'].append(state)
                    top_user_pin_dict['Year'].append(year)
                    top_user_pin_dict['Quarter'].append(int(quarter.removesuffix('.json')))
                    top_user_pin_dict['Pincode'].append(name)
                    top_user_pin_dict['Registered_users'].append(count)
            except:
                pass

top_user_pin_df = pd.DataFrame(top_user_pin_dict)

In [228]:
top_user_pin_df

,State,Year,Quarter,Pincode,Registered_users,Region
0,Andaman and Nicobar Islands,2018,1,744103,1608,Southern Region
1,Andaman and Nicobar Islands,2018,1,744101,1108,Southern Region
2,Andaman and Nicobar Islands,2018,1,744105,1075,Southern Region
3,Andaman and Nicobar Islands,2018,1,744102,1006,Southern Region
4,Andaman and Nicobar Islands,2018,1,744104,272,Southern Region
...,...,...,...,...,...,...
7135,West Bengal,2022,4,700015,108457,Eastern Region
7136,West Bengal,2022,4,742304,105471,Eastern Region
7137,West Bengal,2022,4,721101,105279,Eastern Region
7138,West Bengal,2022,4,700091,102363,Eastern Region


In [229]:
def create_top_user_pincode():
    
    try:
        connection = mysql.connect(
                            host = 'localhost',
                            database = 'phonepe_pulse',
                            user = 'root',
                            password = 'krishaan28920*'
                        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            print('MySQL connected successfully!')
        
        cursor.execute('DROP TABLE IF EXISTS top_user_pincode')

        cursor.execute('''CREATE TABLE IF NOT EXISTS top_user_pincode(
                                            state             VARCHAR(255),
                                            pincode           INT,
                                            year              INT,
                                            quater            VARCHAR(255),
                                            registered_user   INT,
                                            region            VARCHAR(255))'''
                      )

        query = '''
                INSERT INTO top_user_pincode (state, pincode, year, quater, registered_user, region)
                VALUES (%s, %s, %s, %s, %s, %s)

                '''
        top_user_pincode = top_user_pin_df

        for _, row in top_user_pincode.iterrows():
            values = tuple(row)
            cursor.execute(query, values)
            connection.commit()
            
    except Error as e:
        print(f'Error occured: {str(e)}')
        
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print('MySQL connection closed.')

In [230]:
create_top_user_pincode()

MySQL connected successfully!
MySQL connection closed.


# List of dataframes created so far

In [88]:
df_list = [df for df in globals() if isinstance(globals()[df], pd.core.frame.DataFrame) and df.endswith('_df')]

df_list

['agg_trans_df',
 'agg_user_df',
 'map_trans_df',
 'map_user_df',
 'top_trans_dist_df',
 'top_trans_pin_df',
 'top_user_dist_df',
 'top_user_pin_df']

# Columnwise null-count and duplicated_rows-count

In [98]:
for df_name in df_list:
    df = globals()[df_name]
    print(f"{df_name}:")
    print(f"Null count: \n{df.isnull().sum()}")
    print(f"Duplicated rows count: \n{df.duplicated().sum()}")
    print(df.shape)
    print("\n", 25 * "_", "\n")

agg_trans_df:
Null count: 
State                 0
Year                  0
Quarter               0
Transaction_type      0
Transaction_count     0
Transaction_amount    0
Region                0
dtype: int64
Duplicated rows count: 
0
(3594, 7)

 _________________________ 

agg_user_df:
Null count: 
State                0
Year                 0
Quarter              0
Brand                0
Transaction_count    0
Percentage           0
Region               0
dtype: int64
Duplicated rows count: 
0
(6732, 7)

 _________________________ 

map_trans_df:
Null count: 
State                 0
Year                  0
Quarter               0
District              0
Transaction_count     0
Transaction_amount    0
Region                0
dtype: int64
Duplicated rows count: 
0
(14636, 7)

 _________________________ 

map_user_df:
Null count: 
State               0
Year                0
Quarter             0
District            0
Registered_users    0
App_opens           0
Region              0
dtype

# Understanding the dataframes

In [99]:
print('DATAFRAME INFO:\n')

for df_name in df_list:
    df = globals()[df_name]
    print(df_name + ':\n')
    df.info()
    print("\n", 45 * "_", "\n")

DATAFRAME INFO:

agg_trans_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3594 entries, 0 to 3593
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               3594 non-null   object 
 1   Year                3594 non-null   object 
 2   Quarter             3594 non-null   int64  
 3   Transaction_type    3594 non-null   object 
 4   Transaction_count   3594 non-null   int64  
 5   Transaction_amount  3594 non-null   float64
 6   Region              3594 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 196.7+ KB

 _____________________________________________ 

agg_user_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6732 entries, 0 to 6731
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   State              6732 non-null   object 
 1   Year               6732 non-null   obj

# Dropping rows with null values

In [100]:
# 'top_trans_pin_df' seems to have two null values and they are not of significant proportion so dropping them;

top_trans_pin_df.dropna(axis = 'index', inplace = True)
top_trans_pin_df.isnull().sum()

State                 0
Year                  0
Quarter               0
Pincode               0
Transaction_count     0
Transaction_amount    0
Region                0
dtype: int64

# Changing datatype across all dataframes

In [101]:
# Year column in all the dataframes seems to be of object dtype so changing it to int object so as to push into MySQL as year;

for df_name in df_list:
    df = globals()[df_name]
    df['Year'] = df['Year'].astype('int')

# Outlier count across all dataframes

In [102]:
# Everything seems to be alright as far as dtypes and nullvalues are concerned so checking for outliers
# Function to check for outliers

def count_outliers(df):
    outliers = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        if col in ['Transaction_count', 'Transaction_amount']:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            upper_bound = q3 + (1.5 * iqr)
            lower_bound = q1 - (1.5 * iqr)
            outliers[col] = len(df[(df[col] > upper_bound) | (df[col] < lower_bound)])
        else:
            continue
    return outliers

In [103]:
# Everything seems to be alright as far as dtypes and nullvalues are concerned so checking for outliers
# Function to check for outliers

def count_outliers(df):
    outliers = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        if col in ['Transaction_count', 'Transaction_amount']:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            upper_bound = q3 + (1.5 * iqr)
            lower_bound = q1 - (1.5 * iqr)
            outliers[col] = len(df[(df[col] > upper_bound) | (df[col] < lower_bound)])
        else:
            continue
    return outliers

# Unique value count across all dataframes

In [105]:
# Function to check for unique value counts and print if count less than 10;

def unique_value_count(df, exclude_cols=[]):
    for col in df.columns:
        if col in exclude_cols:
            continue
        unique_vals = df[col].nunique()
        print(f"{col}: {unique_vals} unique values")
        if unique_vals < 10:
            print(df[col].unique())

In [106]:
print('UNIQUE VALUE COUNT ACROSS DATAFRAMES; \n')

for df_name in df_list:
    df = globals()[df_name]
    print(df_name, ":\n")
    unique_value_count(df, exclude_cols = ['State', 'Year', 'Quarter', 'Percentage'])
    print("\n", 55 * "_", "\n")

UNIQUE VALUE COUNT ACROSS DATAFRAMES; 

agg_trans_df :

Transaction_type: 5 unique values
['Recharge & bill payments' 'Peer-to-peer payments' 'Merchant payments'
 'Financial Services' 'Others']
Transaction_count: 3548 unique values
Transaction_amount: 3594 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central Region' 'Western Region']

 _______________________________________________________ 

agg_user_df :

Brand: 20 unique values
Transaction_count: 6501 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central Region' 'Western Region']

 _______________________________________________________ 

map_trans_df :

District: 727 unique values
Transaction_count: 14566 unique values
Transaction_amount: 14636 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central Region' 'Western Region'

# Creating CSV files out of the refined dataframes

In [236]:
def save_dfs_as_csv(df_list):
    subfolder = 'Miscellaneous'
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)
        
    for df_name in df_list:
        df = globals()[df_name]
        file_path = os.path.join(subfolder, df_name.replace('_df', '') + '.csv')
        df.to_csv(file_path, index=False)

# Calling function to execute
save_dfs_as_csv(df_list)

# Data Visualization 

In [246]:
year=2018
quater="1"

In [247]:
 total_transactions(year, quater)

,transaction type,transaction amount
1,Recharge & bill payments,1.447260e+10
2,Peer-to-peer payments,1.472451e+11
3,Merchant payments,4.656676e+09
4,Financial Services,8.158531e+08
5,Others,4.643213e+09


In [249]:
top_state_transactions_amount(year, quater)

,state,transaction amount
1,Maharashtra,1.743299e+10
2,Karnataka,1.678323e+10
3,Delhi,1.095004e+10
4,Telangana,1.088046e+10
5,Andhra Pradesh,1.022123e+10
6,Tamil Nadu,9.041592e+09
7,West Bengal,8.797927e+09
8,Uttar Pradesh,7.571870e+09
9,Rajasthan,6.278559e+09
10,Gujarat,5.867998e+09


In [250]:
top_state_transactions_count(year, quater)

,state,transaction count
1,Maharashtra,12326137
2,Karnataka,10177578
3,West Bengal,9033574
4,Andhra Pradesh,7672341
5,Delhi,6942594
6,Telangana,6703014
7,Uttar Pradesh,5980568
8,Tamil Nadu,5373569
9,Rajasthan,5103993
10,Madhya Pradesh,4704960


In [251]:
top_district_transactions_amount(year, quater)

,district,transaction amount
1,Bengaluru Urban,14162750303.06484
2,Pune,5378982326.83592
3,Central Delhi,4312145216.27447
4,Hyderabad,4287223789.76588
5,Chennai,4281137762.52244
6,Mumbai,3923116793.95208
7,Jaipur,2953086960.14137
8,Kolkata,2753347660.14291
9,Rangareddy,2677304926.13757
10,Birbhum,2520486412.45238


In [255]:
top_district_transactions_count(year, quater)

,district,transaction count
1,Bengaluru Urban,8306513
2,Pune,3842929
3,Central Delhi,2997714
4,Mumbai,2671029
5,Chennai,2656230
6,Hyderabad,2545507
7,Jaipur,2417174
8,Birbhum,2125816
9,Kolkata,2043129
10,Bhopal,1704495


In [293]:
top_pincode_transactions_amount(year, quater)

,pincode,transaction amount
1,560001,4.684262e+09
2,110006,3.714139e+09
3,400008,3.105209e+09
4,600003,2.997847e+09
5,731101,2.366771e+09
6,700009,1.851145e+09
7,500012,1.643062e+09
8,411001,1.413524e+09
9,201301,9.876134e+08
10,462001,8.987306e+08


In [294]:
 top_pincode_transactions_count(year, quater)

,pincode,transaction count
1,560001,2917356
2,110006,2636324
3,400008,2227885
4,600003,2010196
5,731101,1764956
6,700009,1390629
7,411001,1055656
8,500012,933013
9,462001,832846
10,201301,690196


In [282]:
user_brand_count(year,quater)

,brand,user count
1,Xiaomi,11926274
2,Samsung,9609356
3,Vivo,5894266
4,Others,5366994
5,Oppo,4479343
6,Realme,2376844
7,Apple,1778928
8,Motorola,1593229
9,OnePlus,1442304
10,Lenovo,1246412


In [283]:
user_brand_count_state(year, quater)

,state,brand,user count
1,Maharashtra,Xiaomi,1606618
2,Maharashtra,Samsung,1354403
3,Uttar Pradesh,Xiaomi,1164619
4,Karnataka,Xiaomi,973827
5,Uttar Pradesh,Samsung,939725
6,Telangana,Xiaomi,853570
7,Andhra Pradesh,Xiaomi,836581
8,Karnataka,Samsung,757979
9,Maharashtra,Vivo,722501
10,Rajasthan,Xiaomi,713005


In [286]:
top_user_state(year, quater)

,state,user count
1,Maharashtra,4224599
2,Andhra Pradesh,2876906
3,Karnataka,2833078
4,Telangana,2530788
5,Uttar Pradesh,2311040
6,Rajasthan,2094534
7,Gujarat,1997209
8,Delhi,1991550
9,West Bengal,1852691
10,Tamil Nadu,1407204


In [288]:
top_user_district(year, quater)

,district,user count
1,Bengaluru Urban,1922368
2,Pune,1211643
3,Jaipur,900773
4,Mumbai Suburban,719300
5,Hyderabad,655175
6,Rangareddy,635175
7,Thane,631864
8,Ahmadabad,572811
9,Medchal Malkajgiri,549429
10,Visakhapatnam,497187


In [296]:
top_user_pincode(year, quater)

,state,pincode,user count
1,Uttar Pradesh,201301,114625
2,Telangana,500072,105012
3,Karnataka,560068,98487
4,Delhi,110059,95496
5,Delhi,110092,83600
6,Haryana,122001,82656
7,Karnataka,560037,75304
8,Gujarat,395006,72275
9,Bihar,800001,71092
10,Rajasthan,302012,68098


In [297]:
import mysql.connector as mysql
from mysql.connector import Error
import pandas as pd


def total_transactions(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT transaction_type, sum(transaction_amount) AS transaction_amount 
                FROM aggregated_transaction 
                WHERE year = %s 
                AND quater = %s 
                GROUP BY transaction_type
                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['transaction type', 
                                           'transaction amount']).reset_index(drop=True)
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
        
def top_state_transactions_amount(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT state, SUM(transaction_amount) AS top_states
                FROM top_transaction
                WHERE year = %s
                AND quater = %s
                GROUP BY state 
                ORDER BY top_states DESC
                LIMIT 10;

                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['state', 
                                           'transaction amount']).reset_index(drop=True)
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
def top_state_transactions_count(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT state, SUM(transaction_count) AS top_states
                FROM top_transaction
                WHERE year = %s
                AND quater = %s
                GROUP BY state 
                ORDER BY top_states DESC
                LIMIT 10;

                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['state', 
                                           'transaction count']).reset_index(drop=True)
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
        
def top_district_transactions_amount(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT district, SUM(transaction_amount) AS top_district_amount
                FROM map_transaction
                WHERE year = %s
                AND quater = %s
                GROUP BY district 
                ORDER BY top_district_amount DESC
                LIMIT 10;

                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['district', 
                                           'transaction amount']).reset_index(drop=True)
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
def top_district_transactions_count(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT district, SUM(transaction_count) AS top_district_count
                FROM map_transaction
                WHERE year = %s
                AND quater = %s
                GROUP BY district 
                ORDER BY top_district_count DESC
                LIMIT 10;

                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['district', 
                                           'transaction count']).reset_index(drop=True)
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
def top_pincode_transactions_amount(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT pincode, SUM(transaction_amount) AS top_pincode_amount
                FROM top_transaction_pincode
                WHERE year = %s
                AND quater = %s
                GROUP BY pincode 
                ORDER BY top_pincode_amount DESC
                LIMIT 10;

                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['pincode', 
                                           'transaction amount']).reset_index(drop=True)
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
def top_pincode_transactions_count(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT pincode, SUM(transaction_count) AS top_pincode_count
                FROM top_transaction_pincode
                WHERE year = %s
                AND quater = %s
                GROUP BY pincode 
                ORDER BY top_pincode_count DESC
                LIMIT 10;

                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['pincode', 
                                           'transaction count']).reset_index(drop=True)
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
        
def user_brand_count(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT brand, SUM(brand_count) AS user_count
                FROM aggregated_user
                WHERE year = %s
                AND quater = %s
                GROUP BY brand
                ORDER BY user_count DESC
                LIMIT 10;

                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['brand', 
                                           'user count'])
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
def user_brand_count_state(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT state, brand, SUM(brand_count) AS state_user_count
                FROM aggregated_user
                WHERE year = %s
                AND quater = %s
                GROUP BY state, brand
                ORDER BY state_user_count DESC
                LIMIT 10;
                
                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['state',
                                           'brand', 
                                           'user count'])
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
def top_user_state(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT state, SUM(registered_user) AS user_count_state
                FROM top_user
                WHERE year = %s
                AND quater = %s
                GROUP BY state
                ORDER BY user_count_state DESC
                LIMIT 10;
    
                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['state',
                                           'user count'])
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
def top_user_district(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT district, SUM(registered_user) AS user_count_state
                FROM top_user
                WHERE year = %s
                AND quater = %s
                GROUP BY district
                ORDER BY user_count_state DESC
                LIMIT 10;
    
                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['district',
                                           'user count'])
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
def top_user_pincode(year, quater):
    try:
        connection = mysql.connect(user='root', 
                                   password='krishaan28920*', 
                                   host='localhost', 
                                   database='phonepe_pulse'
                                )
        cursor = connection.cursor()

        query = '''
                SELECT state, pincode, SUM(registered_user) AS user_count_pincode
                FROM top_user_pincode
                WHERE year = %s
                AND quater = %s
                GROUP BY state, pincode
                ORDER BY user_count_pincode DESC
                LIMIT 10;
    
                '''
        cursor.execute(query, (year, quater))

        result = cursor.fetchall()

        cursor.close()
        connection.close()

        df = pd.DataFrame(result, columns=['state',
                                           'pincode',
                                           'user count'])
        df.index += 1

        return df
    
    except Error as e:
        print(f"Error occurred while connecting to MySQL: {str(e)}")
        
        